In [1]:
import random

In [2]:
text = input()
if text in ["Привет", "Здарова", "Хеллоу"]:
  print(random.choice(["Здрасте", "Йоу", "Приветики"]))
elif text in ["Пока", "Увидимся", "Чао"]:
  print(random.choice(["Буду ждать нашей встречи", "Ок", "Бай-бай"]))
else:
  print("Не понял")

щз
Не понял


In [3]:
# Состав карт "намерений", которые поддерживает наш бот
# examples - фразы на которые реагирует бот
# response - фразы которыми бот отчвечает
INTENTS = {
    "hello": {
        "examples": ["Привет", "Йо", "Хеллоу", "Здравствуйте"],
        "response": ["Здрасте", "Приветик", "Я соскучился"],
    },
    "how_are_you": {
        "examples": ["Как дела", "Как сам", "Чо по настроению", "Ты в ресурсе"],
        "response": ["Дела отлично", "Лучше всех", "Я красавчик и меня очень любит мама"],
    },
}

In [4]:

# Расстояние Левенштейна, мера того насколько строки отличаются
import re
from nltk import edit_distance

# "   ABC   " => "abc"
# "Hello, my friend!!! How are YOU?:)" => "hello my friend how are you"
def filter_text(text):
    # К нижнему регистру "ABC" => "abc"
    text = text.lower()
    # strip - " abc " => "abc"
    text = text.strip()
    # Найти все знаки препинания и заменить их на пустоту ""
    # [^ ] - "набор символов кроме"
    # \w - символы из которых состоят слова (word) = a-z0-9
    # \s - символы пробела (space) = \t \r \n \s
    expression = r"[^\w\s]"
    text = re.sub(expression, "", text)
    return text


def text_match(user_text, example):
    user_text = filter_text(user_text)
    example = filter_text(example)
    distance = edit_distance(user_text, example)
    if distance >= 3:
        return False
    
    # Посчитаем среднюю длину из двух фраз
    avg_len = ( len(example) + len(user_text) ) / 2
    # Процент опечаток
    ratio = distance / avg_len
    if ratio > 0.4:
        print ('false')
        return False # Больше 40% слова опечаталось - значит фразы не совпадают
    else:
        print ('true')
        return True
#    return ration > 0.4
#    reutrn user_text == example

In [5]:
text_match("Константинопольский", "Констатнинопольский")

true


True

In [6]:
text_match("Мир", "Мам")

false


False

In [7]:
edit_distance("Привет", "Чемодан")

7

In [8]:
filter_text("Hello, my friend!!! How are YOU?:)")

'hello my friend how are you'

In [9]:
INTENTS.keys() # Все намерения из карты:

dict_keys(['hello', 'how_are_you'])

In [10]:
INTENTS["hello"] # examples - список фраз примеров, response - список фраз ответов

{'examples': ['Привет', 'Йо', 'Хеллоу', 'Здравствуйте'],
 'response': ['Здрасте', 'Приветик', 'Я соскучился']}

In [11]:
# По фразе пользователя, определить его намерение (интент)
def get_intent(user_text):
    # Для каждого (интента) В INTENTS, сделать следующее:
    for intent in INTENTS:
        examples = INTENTS[intent]["examples"]
        for example in examples:
            if text_match(user_text, example):
                #интент найден
                return intent
    return None

def get_random_response(intent):
    # response, responses
    return random.choice(INTENTS[intent]["responses"])
#делает возврат случайного ответа из карты намерений
#def get_random_response(intent):

In [12]:
get_intent("ты в ресурсе?")

true


'how_are_you'

In [13]:
import json

with open("Desktop/big_bot_config.json", "r") as config_file:
    data = json.load(config_file)

INTENTS = data["intents"]

In [14]:
INTENTS.keys()

dict_keys(['hello', 'bye', 'how_are_you56234', 'how_your_day89347', 'how_are_you_feeling74447', 'do_you_want_play8679435', 'code_phrase978457', 'names', 'mood', 'books', 'films', 'humor', 'ask price', 'operator request', 'name request', 'options request', 'problem', 'hobby', 'singer', 'attractions', 'stories', 'napoleon', 'snoop_dog', 'hot_dog', 'putin', 'santa_barbara', 'bot_name', 'what_cen_you_do', 'thanks', 'help_me', 'not_that', 'how_are_you', 'feedback_good', 'feedback_bad', 'actions', 'hobbies', 'exits', 'abilities', 'jokes', 'comity', 'indetermine', 'news', 'games', 'discount', 'сlosing_account', 'weather', 'pet', 'film', 'name_questions', 'can_questions', 'todo_questions', 'whos_questions', 'bad_questions', 'mood_questions', 'lang_questions', 'whatsup', 'music', 'gifts', 'cities', 'constellation', 'chess', 'byflood', 'electronics', 'skills', 'reactions_to_bad_feelings', 'reactions_to_good_feelings', 'gender', 'what_are_you_doing', 'hellozdr', 'howtonlkt', 'pythonbooks', 'howar

In [15]:
INTENTS["news"]

{'examples': ['Что происходит', 'Что слышно', 'Что новенького', 'Новости'],
 'responses': ['Все ждут перемен',
  'Пока всё тихо',
  'Готовимся к Новому году',
  'Скоро зима']}

In [16]:
# Подготовить данные: обучающая выборка 
# Фраза(X) + интент y)
X = [] 
y = []

for intent in INTENTS:
    examples = INTENTS[intent]["examples"]
    for example in examples:
        text = filter_text(example)
        if len(text) < 3:
            continue # Пропускаем неподходящий пример
        X.append(text)
        y.append(intent)

In [17]:
#print (X[123], y[123])
len(X)

2979

In [18]:
#Векторизация текстов
#"текст" => вектор [1,2,3,4]
#Например: CountVectorizer, TfIDFVectorizer

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()  # Настройки
vectorizer.fit(X) # Обучаем векторайзер

vecX = vectorizer.transform(X)  # Сохраняем векторизованные тексты

In [20]:
X[505]

'чем ты занимаешь в свободное время'

In [21]:
y[505]

'hobbies'

In [22]:
print (vecX[505])

  (0, 395)	1
  (0, 740)	1
  (0, 1933)	1
  (0, 2254)	1
  (0, 2450)	1


In [23]:
print(vectorizer.transform(["чем ты занимаешь время в свободное время чем"]))

  (0, 395)	2
  (0, 740)	1
  (0, 1933)	1
  (0, 2254)	1
  (0, 2450)	2


In [24]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()  # настройки
model.fit(vecX, y)  # Обучаем модель

RandomForestClassifier()

In [25]:
INTENTS["hobbies"]["examples"]

['чтение',
 'еда',
 'спорт',
 'рисование',
 'игры',
 'музыка',
 'вышивка',
 'прогулки',
 'просмотр кино',
 'поход в кино',
 'шитье',
 'сон',
 'расскажи о своих хобби',
 'видеоигры',
 'Чем ты занимаешь в свободное время',
 'У тебя есть любимые занятия']

In [26]:
model.predict(vectorizer.transform(["примеры кода"])) # Сделать предсказание, по фразе определить интент

array(['sixth_asking'], dtype='<U29')

In [27]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(vecX)  # Сделаем предсказание на обучающей выборке

print("[TRAIN] accuracy_score = ", accuracy_score(y, y_pred))
print("[TRAIN] f1_score = ", f1_score(y, y_pred, average="macro"))

[TRAIN] accuracy_score =  0.829137294394092
[TRAIN] f1_score =  0.7555922313814873


In [28]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(vecX, y)
mlp_pred = mlp.predict(vecX)
print("[TRAIN] accuracy_score = ", accuracy_score(y, mlp_pred))
print("[TRAIN] f1_score = ", f1_score(y, mlp_pred, average="macro"))

[TRAIN] accuracy_score =  0.8288016112789527
[TRAIN] f1_score =  0.7531747839753618


/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [29]:
# get_intent(text)
# get_random_response(intent)

# Определяем интент по тексту с помощью модели
def get_intent_ml(text):
    vec_text = vectorizer.transform([text])
    pred_intent = model.predict(vec_text)[0]
    # model.predict_proba() < 30% ? => random.choice(fail_phrases)
    return pred_intent

def bot(text):
    intent = get_intent(text)
    if intent:
        return get_random_response(intent)

    intent = get_intent_ml(text)
    return get_random_response(intent)

In [30]:
bot ("Как твои дела дружище?")

'Хорошо'

In [31]:
user_text = ""
while user_text != "exit":
    print("[USER]: ", end='')
    user_text = input()
    reply = bot(user_text)
    print(f"[BOT]: {reply}")

[USER]: рпивет
true
[BOT]: Рад встрече
[USER]: как дела?
true
[BOT]: Рад знакомству
[USER]: учше
false
[BOT]: Это действительно радостно
[USER]: чуше
false
false
[BOT]: Это действительно радостно
[USER]: exit
true
[BOT]: Уже уходите? Ладно, пока...


In [ ]:
! pip list 

In [ ]:
! pip install python-telegram-bot --pre

In [ ]:
# ASYNCIO FIX?

In [36]:
TOKEN = ""

In [37]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from telegram import Update # (единица информации полученная с сервера ТГ)
from telegram.ext import ApplicationBuilder, MessageHandler, filters

# TOKEN <= @BotFather
app = ApplicationBuilder().token(TOKEN).build()

async def reply_by_bot(upd: Update, _ctx):
    text = upd.message.text
    print(f"[{upd.effective_user.full_name}]: {text}")
    reply = bot(text)
    print(f"[BOT]: {reply}")
    await upd.message.reply_text(reply)

# CommandHandler <= /info
# MessageHandler <= "привет, бот"
handler = MessageHandler(filters.TEXT,reply_by_bot)
app.add_handler(handler)

app.run_polling(poll_interval=1)


[Vladimir]: /start
false
[BOT]: Ваш успех радует
[Vladimir]: привет
true
[BOT]: доброго времени суток
[Vladimir]: как дела?
true
[BOT]: БОЛЬШЕ ЧЕРЕПОВ ДЛЯ ТРОНА!!!
КРОВЬ ДЛЯ БОГА КРОВИ!!!
КРОВЬ ДЛЯ БОГА КРОВИ!!!
[Vladimir]: ужас
true
[BOT]: У всех нас бывают плохие дни. Что бы вы хотели поделать сейчас?
[Vladimir]: поговорить
true
[BOT]: Не люблю пустые разговоры
[Vladimir]: о жизни
[BOT]: Это действительно радостно
[Vladimir]: что делаешь?
true
[BOT]: Возвращаяйтесь скорее
[Vladimir]: я ещё здесь
[BOT]: Это действительно радостно
[Vladimir]: exit
true
[BOT]: I`m out!
[Vladimir]: Привет
true
[BOT]: Хэй
[Vladimir]: Exit
true
[BOT]: Увидимся


Error while getting Updates: httpx HTTPError: [Errno 8] nodename nor servname provided, or not known
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/anyio/_core/_sockets.py", line 149, in connect_tcp
    addr_obj = ip_address(remote_host)
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/ipaddress.py", line 53, in ip_address
    raise ValueError('%r does not appear to be an IPv4 or IPv6 address' %
ValueError: 'api.telegram.org' does not appear to be an IPv4 or IPv6 address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/httpcore/_exceptions.py", line 8, in map_exceptions
    yield
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/httpcore/backends/asyncio.py", line 109, in connect_tcp
    stream: anyio.abc.B

[Vladimir]: Привет
true
[BOT]: Здравствуй, дорогой друг!


Error while getting Updates: httpx HTTPError: [Errno 8] nodename nor servname provided, or not known
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/anyio/_core/_sockets.py", line 149, in connect_tcp
    addr_obj = ip_address(remote_host)
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/ipaddress.py", line 53, in ip_address
    raise ValueError('%r does not appear to be an IPv4 or IPv6 address' %
ValueError: 'api.telegram.org' does not appear to be an IPv4 or IPv6 address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/httpcore/_exceptions.py", line 8, in map_exceptions
    yield
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/httpcore/backends/asyncio.py", line 109, in connect_tcp
    stream: anyio.abc.B

[Vladimir]: Как дела?
true
[BOT]: Приветик в этом чатике))


Error while getting Updates: httpx HTTPError: [Errno 8] nodename nor servname provided, or not known
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/anyio/_core/_sockets.py", line 149, in connect_tcp
    addr_obj = ip_address(remote_host)
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/ipaddress.py", line 53, in ip_address
    raise ValueError('%r does not appear to be an IPv4 or IPv6 address' %
ValueError: 'api.telegram.org' does not appear to be an IPv4 or IPv6 address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/httpcore/_exceptions.py", line 8, in map_exceptions
    yield
  File "/Users/vladimirpolynskii/opt/anaconda3/lib/python3.9/site-packages/httpcore/backends/asyncio.py", line 109, in connect_tcp
    stream: anyio.abc.B